In [ ]:
#see temp

In [1]:
import cv2

In [2]:
import numpy as np

def show(img):
    cv2.imshow('title', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

#to read
orig = cv2.imread('C:/Users/PC/Codes/mosaic.jpg')
orig = cv2.resize(orig, (500, 500))
image = orig.copy()

In [22]:
#dist btw two parallel lines d = |c1–c2|/√(1+m^2).
class ControlLine:
    
    def __init__(self, m, c, speed=1):
        self.speed = speed
        self.m = m
        self.c = c
    
    def update_c(self):   
        #updates c, so that more close to corner point, speed proprotional to slope
        #dist btw two parallel lines d = |c1–c2|/√(1+m^2)
        
        self.c = self.speed * np.sqrt(1 + self.m**2) + self.c
        return self.c

In [29]:
def apply_filter(filtr, back, mask): 
    #filter, background, mask(roi for filter)
    r1 = cv2.bitwise_and(filtr, filtr, mask=mask) #roi to apply filter
    mask_inv = cv2.bitwise_not(mask)
    r2 = cv2.bitwise_and(back, back, mask=mask_inv) #no filter
    result = cv2.addWeighted(r1, 1, r2, 1, 0)
    return result

In [32]:

def move_line(image, m):
    orig = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    result = orig #image, which is modified, on which filter is applied
    base = np.zeros(orig.shape, np.uint8) #on this the white line is drawn
    threshold = np.array((255, 255, 255))
    x, y, _ = base.shape
    m = -abs(m) #to keep it negative
    if m==0:
        m = 1e-4
    last_c = int(y - m*x)
    assert base.shape == orig.shape == gray.shape
    
    controller = ControlLine(m, 0, 2)
    c =  controller.update_c()
    while controller.update_c() < (last_c):
        c =  controller.update_c()
        img = base.copy() #so that only line portion is modified, not whole again, thus wasting
        y1 = int(m*0 + c)
        x1 = int(-c / m)
        cv2.line(img, (x1, 0), (0, y1), (255, 255, 255), 10)
      
        mask = cv2.inRange(img, threshold, threshold)
        result = apply_filter(gray, result, mask)
        cv2.imshow('t', result)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()
    

# move_line(image, -0.5)
# t = np.zeros((512,512,3), np.uint8)
# move_line(t, -0)

In [35]:
#to draw lines

pts = {}
line_drawn = False

def slope(pts):
    (x1, y1), (x2, y2) = pts.values()
    if x1 == x2:
        x2 += 0.01
    return (y2 - y1) / (x2 - x1)

def click_and_crop(event, x, y, flags, param):
    global pts, line_drawn
    
    if event == cv2.EVENT_LBUTTONDOWN:
        pts['start'] = (x, y)
        
    elif event == cv2.EVENT_LBUTTONUP:
        line_drawn = True
        pts['end'] = (x, y)
        cv2.line(image, pts['start'], pts['end'], (0, 255, 0), 2)
        cv2.imshow("image", image)

clone = orig.copy()
image = clone.copy()
cv2.namedWindow("image")
cv2.setMouseCallback("image", click_and_crop) #To process the mouse click events we define the 
# click_and_crop  callback function on Line 10. Anytime a mouse event happens, OpenCV will relay the 
# pertinent details to our click_and_crop  function.

# keep looping until the 'q' key is pressed
while not line_drawn:
    # display the image and wait for a keypress
    cv2.imshow("image", image)
    key = cv2.waitKey(1)
    
cv2.destroyAllWindows()
move_line(image, slope(pts))
cv2.destroyAllWindows()

In [ ]:
slope(pts)

In [ ]:
pts.values()

In [ ]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
show(gray)

In [ ]:
gray3 = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
show(gray3)

In [ ]:
gray3.shape

In [ ]:
#to show and color portion of image

zeros = np.zeros(img.shape[:2], dtype='uint8')
colors = ['b', 'g', 'r']
clridx = colors.index(input("Choose any color Blue('b') Green('g') Red('r')").lower())
clrimg = [zeros, zeros, zeros]
clrimg[clridx] = cv2.split(image)[clridx] 
show(cv2.merge(clrimg))

In [ ]:
cc = np.ones(img.shape[:2], dtype='uint8') * 255
pts = np.array([[10,5],[20,30],[70,20],[50,10]], np.int32)
pts = pts.reshape((-1,1,2))
cv2.polylines(cc,[pts],True,(0,0,0), 50)
show(cc)

In [ ]:
t = np.zeros((512,512,3), np.uint8)

cv2.line(t,(0,100),(100, 0),(255,255,0),10)
show(t)

In [ ]:
def calc_slope(pts1, pts2):
    (x1, y1), (x2, y2) = pts1, pts2
    return (y2 - y1) / (x2 - x1)

In [ ]:
import cv2
import numpy as np

def move_line(img, m):
    x, y, _ = img.shape
    last_c = int(y - m*x)
    arr = np.array((255, 255, 255))
    for c in range(last_c):
        y1 = int(m*0 + c)
        x1 = int(-c / m)
        cv2.line(img, (x1, 0), (0, y1), (255, 255, 255), 10)
        
        mask = cv2.inRange(img, arr, arr)
        m1 = cv2.bitwise_and(image, image, mask=cv2.bitwise_not(mask))
        m2 = cv2.bitwise_and(gray, gray, mask=(mask))
        result = cv2.addWeighted(m1, 1, m2, 1, 0)
        cv2.imshow('t', result)
        cv2.waitKey(1)
    cv2.destroyAllWindows()
    
t = np.zeros((512,512,3), np.uint8)
# image = cv2.resize(image, (512, 512))
# gray = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
move_line(t, -0.5)


In [ ]:
gray.shape

In [ ]:
show(image)

In [ ]:
#to draw lines

pts = {}
line_drawn = False

def click_and_crop(event, x, y, flags, param):
    global pts, line_drawn
    
    if event == cv2.EVENT_LBUTTONDOWN:
        pts['start'] = (x, y)
        
    elif event == cv2.EVENT_LBUTTONUP:
        line_drawn = True
        pts['end'] = (x, y)
        cv2.line(image, pts['start'], pts['end'], (0, 255, 0), 2)
        cv2.imshow("image", image)

clone = orig.copy()
image = clone.copy()
cv2.namedWindow("image")
cv2.setMouseCallback("image", click_and_crop) #To process the mouse click events we define the 
# click_and_crop  callback function on Line 10. Anytime a mouse event happens, OpenCV will relay the 
# pertinent details to our click_and_crop  function.

# keep looping until the 'q' key is pressed
while not line_drawn:
    # display the image and wait for a keypress
    cv2.imshow("image", image)
    key = cv2.waitKey(1)
    

        
cv2.destroyAllWindows()

In [ ]:
def calc_slope(pts1, pts2):
    (x1, y1), (x2, y2) = pts1, pts2
    return (y2 - y1) / (x2 - x1)